In [1]:
#import packages

import rasterio.features
import geopandas as gpd
import pandas as pd
from sklearn.metrics import confusion_matrix
import seaborn as sns
# import rasterstats as rs
# import pyproj
# from pyproj import Transformer
# import fiona
# import random
# import scikitlearn
# import sklearn
# import rasterio
import numpy as np
import geowombat as gw
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import matplotlib.pyplot as plt

In [2]:
# #define satellite image, aoi, and training data paths
# raster = "/Users/philipp/BPLA Dropbox/03 Planning/1232-T2-GIS and Remote Sensing-TMO/02_GIS_Data/LandCoverClassificaiton/lcc-trial/21may-lcc.tif"
# raster="/Users/philipp/BPLA Dropbox/08 GIS-data/GIS-data-Riyadh/2022-08-DQ-PNEO-satellite-image/geoserve_orthos-saudi-arabia_2023-02-22_1207/oDIM_PNEO4_202208080738351_PMS-FS_SEN_PWOI_000072103_2_1_F_1.tif"
raster="/Users/philipp/BPLA Dropbox/03 Planning/1297-Makkah-Environmental-Sustainability-Strategy/02-Data/02-satellite/Makkah-WV2-ST-2023_I602488/Makkah-WV2-ST-2023-MOS/23JAN19075627-S2AM-Makkah-WV2-ST-2023-tiled.png"
# aoi = "/Users/philipp/BPLA Dropbox/03 Planning/1232-T2-GIS and Remote Sensing-TMO/02_GIS_Data/LandCoverClassificaiton/lcc-trial/aoi-lcc.geojson"
training = "/Users/philipp/BPLA Dropbox/03 Planning/1297-Makkah-Environmental-Sustainability-Strategy/02-Data/03-poc/mekka-training-tree-0.shp"

In [3]:
# read training polygones
train = gpd.read_file(training) # this may contain multipolygons
train = train.to_crs(32637)
train_ex = train.explode()
train_ex.geometry=train_ex.centroid # we should use the full range of values inside the polygon not jsust the centroid 
# train_ex['lon'] = train_ex.centroid.x  
# train_ex['lat'] = train_ex.centroid.y
# TODO: https://www.matecdev.com/posts/random-points-in-polygon.html
# aoi = geopandas.read_file(aoi)

In [ ]:
with rasterio.open(raster, 'r') as ds:
    arr = ds.read()  # read all raster values
    transform = ds.transform 
    crs = ds.crs
    res=ds.res

print(arr.shape)
print(crs)
print(res)
print(transform)

In [5]:
# Transform the CRS to WGS84 lat/lon
train_ex = train_ex.to_crs('epsg:32637')
# 
# with gw.open(raster) as src:
#     df = src.gw.extract(train_ex)

with gw.config.update(sensor='bgr'):
    with gw.open(raster) as src:
        df = src.gw.extract(
            train_ex,
            band_names=src.band.values.tolist()
        )

11:03:07:ERROR:555:api.__init__:  The file format is not recognized.
NoneType: None


OSError: 

In [ ]:

import numpy as np
raster =Image.open('image.tif')

In [ ]:
src = rasterio.open(raster)

src.is_tiled
src.block_shapes

In [ ]:
import dask_image.imread

In [ ]:
ortho=dask_image.imread.imread(raster)

In [ ]:
data = np.array(ortho)

In [ ]:
ortho.shape

In [ ]:
with rasterio.open(raster, 'r') as ds:
    arr = ds.read()  # read all raster values
    transform = ds.transform 
    crs = ds.crs
    res=ds.res

print(arr.shape)
print(crs)
print(res)
print(transform)

In [ ]:
# src = rasterio.open(raster)
# transform = src.transform 
# crs = src.crs
# array = src.read()
# print(crs)
# print(src.res)

In [ ]:
# Transform the CRS to WGS84 lat/lon
train_ex = train_ex.to_crs('epsg:32637')

# with gw.open(raster) as src:
#     df = src.gw.extract(train_ex)
with gw.config.update(sensor='bgr'):
    with gw.open(raster) as src:
        df = src.gw.extract(
            train_ex,
            band_names=src.band.values.tolist()
        )

In [ ]:
df

In [ ]:
# Assign class labels based on your training data
target = df['ano'].values
data_in = df[["blue","red","green"]]

# Split the data into training and validation sets
# from scikit learn package, we use the function train_test_split 
X_train, X_val, y_train, y_val = train_test_split(data_in, target, test_size=0.2, random_state=42)#, stratify=target)

In [ ]:
# create a Random Forest Classifier instance
rf = RandomForestClassifier(n_estimators=100, max_depth=10, random_state=42, oob_score=True)

# fit the model to our training data
rf.fit(X_train, y_train)

# predict on validataion data
predicted = rf.predict(X_val)

In [ ]:
# accuracy 
accuracy = accuracy_score(y_val, predicted)
print(f'Out-of-bag score estimate: {rf.oob_score_:.3}')
print(f'Mean accuracy score: {accuracy:.3}')

In [ ]:
cm = pd.DataFrame(confusion_matrix(y_val, predicted), columns=df.ano.unique(), index=df.ano.unique())
sns.heatmap(cm, annot=True)

In [ ]:
#combine bands into a single numpy array
band1 = src.read(1)
band2 = src.read(2)
band3 = src.read(3)
bands = np.array([band1, band2, band3])

In [ ]:
rasterpd = pd.DataFrame(bands.reshape([3,-1]).T)

In [ ]:
rasterpd.head()

In [ ]:
pred_all = rf.predict(rasterpd)

In [ ]:
pred_all.shape

In [ ]:
pred_raster_int = np.unique(pred_all, return_inverse = True)[1]  

In [ ]:
pred_raster=pred_raster_int.reshape([15568,13513])

In [ ]:
from rasterio.plot import show_hist
from matplotlib import pyplot
from rasterio.plot import show
fig, (axrgb, axhist) = pyplot.subplots(1, 2, figsize=(14,7))
show(pred_raster, ax=axrgb)
show_hist(pred_raster, bins=50, histtype='stepfilled',
          lw=0.0, stacked=False, alpha=0.3, ax=axhist)
pyplot.show()

In [ ]:
# predict_df = pd.DataFrame(pred_all)

In [ ]:
# Define the export parameters
#export_params is a python dictionary
export_params = {
    'image': pred_raster,
    'driver': 'GTiff',
    'height': src.shape[0],
    'width': src.shape[1],
    'count': 1,
    'dtype': 'int8',
    'crs': src.crs,
    'transform': src.transform,
}

# Save the classified image as a GeoTIFF 
with rasterio.open("lcc_classification1.tif", "w", **export_params) as dst:
    dst.write(pred_raster, 1)

In [ ]:
Model_apply_predict_df = pd.DataFrame(pred_all)
#https://gis.stackexchange.com/questions/362804/calculate-new-band-for-satellite-image-after-predict-values-with-rf